In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import dataloader
import matplotlib.pyplot as plt


In [2]:
DATA, SAMPLE_LENGTH, SAMPLE_RATE = dataloader.load_data()

...
[0/4096]
[1/4096]
[2/4096]
[3/4096]
[4/4096]
[5/4096]
[6/4096]
[7/4096]
[8/4096]
[9/4096]
[10/4096]
[11/4096]
[12/4096]
[13/4096]
[14/4096]
[15/4096]
[16/4096]
[17/4096]
[18/4096]
[19/4096]
[20/4096]
[21/4096]
[22/4096]
[23/4096]
[24/4096]
[25/4096]
[26/4096]
[27/4096]
[28/4096]
[29/4096]
[30/4096]
[31/4096]
[32/4096]
[33/4096]
[34/4096]
[35/4096]
[36/4096]
[37/4096]
[38/4096]
[39/4096]
[40/4096]
[41/4096]
[42/4096]
[43/4096]
[44/4096]
[45/4096]
[46/4096]
[47/4096]
[48/4096]
[49/4096]
[50/4096]
[51/4096]
[52/4096]
[53/4096]
[54/4096]
[55/4096]
[56/4096]
[57/4096]
[58/4096]
[59/4096]
[60/4096]
[61/4096]
[62/4096]
[63/4096]
[64/4096]
[65/4096]
[66/4096]
[67/4096]
[68/4096]
[69/4096]
[70/4096]
[71/4096]
[72/4096]
[73/4096]
[74/4096]
[75/4096]
[76/4096]
[77/4096]
[78/4096]
[79/4096]
[80/4096]
[81/4096]
[82/4096]
[83/4096]
[84/4096]
[85/4096]
[86/4096]
[87/4096]
[88/4096]
[89/4096]
[90/4096]
[91/4096]
[92/4096]
[93/4096]
[94/4096]
[95/4096]
[96/4096]
[97/4096]
[98/4096]
[99/4096]
[100/4

In [3]:
CTX_SIZE = 500
SAMPLE_GEN = 400

In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.conv_1 = nn.Conv1d(1, 100, 50)
        self.conv_2 = nn.Conv1d(100, 50, 50)
        self.conv_3 = nn.Conv1d(50, 10, 50)
        self.conn   = nn.Linear(1150, 500)

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)

        x = self.conv_1(x)
        x = nn.MaxPool1d(5)(x)
        x = self.conv_2(x)
        x = nn.MaxPool1d(5)(x)
        x = self.conv_3(x)
        x = nn.MaxPool1d(5)(x)
        
        x = x.view(x.shape[0], -1)
        x = self.conn(x)

        return x

enc = Encoder()

In [5]:
x = Variable(torch.rand(1, 1, SAMPLE_LENGTH))

In [6]:
ctx = enc(x)

In [29]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.last_proc_1 = nn.Conv1d(1, 100, 10)
        self.last_proc_2 = nn.Linear((SAMPLE_GEN - 9) * 100, 100)
        self.ctx_proc  = nn.Linear(CTX_SIZE, 100)
        self.proc_both = nn.Linear(200, 500)
        self.proc_out  = nn.Linear(500, SAMPLE_GEN)
        self.proc_ctx  = nn.Linear(500, CTX_SIZE)

    def forward(self, last, ctx):
        last = last.view(last.shape[0], 1, -1)
        last = self.last_proc_1(last)
        last = last.view(last.shape[0], -1)
        last = self.last_proc_2(last)
        ctx = self.ctx_proc(ctx)

        combined = torch.cat((last, ctx), dim=1)
        proc = self.proc_both(combined)
        out = self.proc_out(proc)
        new_ctx = self.proc_ctx(proc)

        return out, new_ctx

dec = Decoder()

In [30]:
last = Variable(torch.rand(1, SAMPLE_GEN))

In [31]:
dec(last, ctx)

(Variable containing:
 
 Columns 0 to 9 
  0.0221  0.0311  0.0675 -0.0659  0.0363 -0.0106  0.0015  0.0143  0.0498 -0.0085
 
 Columns 10 to 19 
 -0.0471  0.0449  0.0260 -0.0295 -0.1347  0.0489  0.0983 -0.0272 -0.0695 -0.0367
 
 Columns 20 to 29 
 -0.0425 -0.0057  0.0567 -0.0789 -0.0671 -0.0097 -0.0754  0.0007 -0.0160 -0.0770
 
 Columns 30 to 39 
 -0.0761  0.0028 -0.0088 -0.0695 -0.0743  0.0041  0.0150 -0.0139  0.0602 -0.0141
 
 Columns 40 to 49 
  0.0027  0.0285  0.0259 -0.0802 -0.0508  0.0870  0.0657 -0.1060 -0.0366  0.0850
 
 Columns 50 to 59 
 -0.0308  0.0390 -0.0062 -0.1018  0.0478  0.0096  0.0553 -0.0639  0.0145 -0.0083
 
 Columns 60 to 69 
  0.0099  0.0012  0.0661  0.0185 -0.0937  0.0488 -0.0392 -0.0780  0.0394  0.0652
 
 Columns 70 to 79 
  0.0375 -0.0286  0.0018  0.0382  0.0271  0.0678 -0.0335  0.1591  0.0003  0.0080
 
 Columns 80 to 89 
  0.0001  0.0350  0.1037  0.0470  0.0552  0.0653  0.0020 -0.0150  0.0526 -0.0602
 
 Columns 90 to 99 
 -0.0347 -0.0340 -0.0687  0.0054 -0.0869 